In [ ]:
from ase.io import write
from ase.io import Trajectory
from ase.visualize import view
from ase.md.langevin import Langevin
from ase import units

from fairchem.core import pretrained_mlip, FAIRChemCalculator

from rdkit import Chem
from rdkit.Chem import AllChem
from ase import Atoms

Molecular Dynamics

In [ ]:
# 1. tert-butanolのSMILESを作成
smiles = "CC(C)(C)O"
rdmol = Chem.MolFromSmiles(smiles)
rdmol = Chem.AddHs(rdmol)

# 2. 3D座標を生成
AllChem.EmbedMolecule(rdmol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(rdmol)

# 3. 原子の位置と記号を抽出
conf = rdmol.GetConformer()
positions = np.array([list(conf.GetAtomPosition(i)) for i in range(rdmol.GetNumAtoms())])
symbols = [atom.GetSymbol() for atom in rdmol.GetAtoms()]

# 4. ASE Atomsオブジェクトを作成
mol = Atoms(symbols=symbols, positions=positions)

# 5. 真空パディングを追加
mol.center(vacuum=5.0)
mol.pbc = (False, False, False)

# 6. .trajに保存
write("tBuOH_pre_md.traj", mol)

In [5]:
# Check before running MD
traj = Trajectory('tBuOH_pre_md.traj')
view(traj) 

<Popen: returncode: None args: ['/Users/lpc_0066/anaconda3/envs/fairchem_env...>

In [9]:
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cpu")
calc = FAIRChemCalculator(predictor, task_name="omol")

# atoms = molecule("H2O")
mol.calc = calc

dyn = Langevin(
    mol,
    timestep=0.1 * units.fs,
    temperature_K=400,
    friction=0.001 / units.fs,
)
trajectory = Trajectory("tBUOH_post_md.traj", "w", mol)
dyn.attach(trajectory.write, interval=1)
dyn.run(steps=1000)

True

In [11]:
# Check after running MD
traj_md = Trajectory('tBUOH_post_md.traj')
view(traj_md) 

<Popen: returncode: None args: ['/Users/lpc_0066/anaconda3/envs/fairchem_env...>

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/lpc_0066/anaconda3/envs/fairchem_env/lib/python3.11/site-packages/ase/gui/pipe.py", line 34, in <module>
    main()
  File "/Users/lpc_0066/anaconda3/envs/fairchem_env/lib/python3.11/site-packages/ase/gui/pipe.py", line 30, in main
    plt.show()
  File "/Users/lpc_0066/anaconda3/envs/fairchem_env/lib/python3.11/site-packages/matplotlib/pyplot.py", line 614, in show
    return _get_backend_mod().show(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/lpc_0066/anaconda3/envs/fairchem_env/lib/python3.11/site-packages/matplotlib_inline/backend_inline.py", line 90, in show
    display(
  File "/Users/lpc_0066/anaconda3/envs/fairchem_env/lib/python3.11/site-packages/IPython/core/display_functions.py", line 245, in display
    print(*objs)
ValueError: I/O operation on closed file.
